In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
from agent_api.agent import GroqAgent

In [ ]:
agent = GroqAgent(
    name="my_test_agent",
    mcp_server="http://127.0.0.1:9090/mcp",
    active_tools=["process_video", "get_video_clip_from_image"],
)
await agent.setup()

In [ ]:
print(agent.routing_system_prompt)

In [ ]:
print(agent.tool_use_system_prompt)

In [ ]:
print(agent.general_system_prompt)

In [ ]:
agent._route_query("Hello, I'm Miguel!")

In [ ]:
agent._route_query("Get me the clip when the character says 'Hello, I'm Miguel!'")

In [ ]:
await agent.chat("Yoooo", "./videos/pass_the_butter_rick_and_morty.mp4")

In [ ]:
await agent.chat(
    "In the video, can you tell me what is the color of the table?",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

In [ ]:
await agent.chat(
    "Give me a clip of the video where Rick says 'Pass the butter'",
    "./videos/pass_the_butter_rick_and_morty.mp4",
)

In [ ]:
agent.reset_memory()